In [10]:
import os
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
from pathlib import Path

from python import classes

In [11]:
input_dir = "crop"
output_dir = "data"
classification = "types"  # TODO
cv_folds = 5

In [12]:
assert classification in ['models', 'types'], "classification must be one of ['models', 'types']"

In [13]:
output_dir = Path(output_dir) / f"{classification}"

In [14]:
# Gather file paths for each class
data = defaultdict(list)
for model_dir in os.listdir(input_dir):
    class_path = os.path.join(input_dir, model_dir)
    if os.path.isdir(class_path):
        for img_file in os.listdir(class_path):
            key = classes.classes_map[classification][model_dir]
            data[key].append(os.path.join(class_path, img_file))
            
df = pd.DataFrame(map(lambda kv: (kv[0], len(kv[1])), data.items()), columns=['model', 'total'])
df.set_index('model', inplace=True)

print(f"Classes found: {len(df)}")

Classes found: 2


In [15]:
# Create data and label arrays
file_paths = []
labels = []
for label, paths in data.items():
    file_paths.extend(paths)
    labels.extend([label] * len(paths))

file_paths = np.array(file_paths)
labels = np.array(labels)

In [16]:
# Stratified K-Fold cross-validation
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

shutil.rmtree(output_dir, ignore_errors=True)
for fold_idx, (_, test_idx) in enumerate(skf.split(file_paths, labels)):
    fold_path = os.path.join(output_dir, f"cv{fold_idx + 1}")
    os.makedirs(fold_path, exist_ok=True)
    df[f"cv{fold_idx + 1}"] = 0

    for idx in test_idx:
        class_label = labels[idx]
        class_fold_path = os.path.join(fold_path, class_label)
        os.makedirs(class_fold_path, exist_ok=True)
        shutil.copy(file_paths[idx], class_fold_path)
        df.at[f"{class_label}", f"cv{fold_idx + 1}"] += 1 
    print(f"cv{fold_idx + 1} completed.")

cv1 completed.
cv2 completed.
cv3 completed.
cv4 completed.
cv5 completed.


In [17]:
df

,total,cv1,cv2,cv3,cv4,cv5
model,,,,,,
airplane,29572,5915,5915,5914,5914,5914
helicopter,2345,469,469,469,469,469


In [18]:
df_sum = df.sum()
df_sum

total    31917
cv1       6384
cv2       6384
cv3       6383
cv4       6383
cv5       6383
dtype: int64